In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 30.7 MB/s eta 0:00:00


In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time
import re

options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

all_data = []
max_pages = 25
page_count = 1

try:
    driver.get("https://www.rumah123.com/jual/rumah/?maxPrice=5000000000&minPrice=1000000000&q=dki%20jakarta")
    time.sleep(5)

    while page_count <= max_pages:
        print(f"📄 Scraping halaman {page_count}...")
        time.sleep(3)

        cards = driver.find_elements(By.CLASS_NAME, "card-featured__middle-section")

        for card in cards:
            try:
                harga_elem = card.find_element(By.CSS_SELECTOR, ".card-featured__middle-section__price > strong")
                harga = harga_elem.text
            except:
                harga = ""

            try:
                info_items = card.find_elements(By.CSS_SELECTOR, ".relative.ui-molecules-list__item")
                kt = info_items[0].text if len(info_items) > 0 else ""
                km = info_items[1].text if len(info_items) > 1 else ""
                garasi = info_items[2].text if len(info_items) > 2 else ""
            except:
                kt = km = garasi = ""

            try:
                info_tags = card.find_elements(By.CLASS_NAME, "attribute-info")
                info_text = " ".join(tag.text for tag in info_tags)
            except:
                info_text = ""

            lt = re.search(r"LT[:\s]+(\d+)", info_text)
            lb = re.search(r"LB[:\s]+(\d+)", info_text)

            all_data.append({
                "KT": kt,
                "KM": km,
                "Garasi": garasi,
                "LT": lt.group(1) if lt else "",
                "LB": lb.group(1) if lb else "",
                "Harga": harga,
            })

        # Naik ke halaman berikutnya
        try:
            next_btn = driver.find_element(By.CLASS_NAME, "ui-molecule-paginate__item--next")
            if "disabled" in next_btn.get_attribute("class"):
                break  # Tidak bisa klik lagi
            driver.execute_script("arguments[0].click();", next_btn)
            page_count += 1
        except (NoSuchElementException, ElementClickInterceptedException):
            break

    # Simpan ke CSV
    df = pd.DataFrame(all_data)
    cols = [col for col in df.columns if col != "Harga"] + ["Harga"]
    df = df[cols]
    df.to_csv("data_rumah123.csv", index=False, encoding="utf-8-sig")
    print("✅ Data dari 25 halaman berhasil disimpan ke 'data_rumah123.csv'")

finally:
    driver.quit()


📄 Scraping halaman 1...
📄 Scraping halaman 2...
📄 Scraping halaman 3...
📄 Scraping halaman 4...
📄 Scraping halaman 5...
📄 Scraping halaman 6...
📄 Scraping halaman 7...
📄 Scraping halaman 8...
📄 Scraping halaman 9...
📄 Scraping halaman 10...
📄 Scraping halaman 11...
📄 Scraping halaman 12...
📄 Scraping halaman 13...
📄 Scraping halaman 14...
📄 Scraping halaman 15...
📄 Scraping halaman 16...
📄 Scraping halaman 17...
📄 Scraping halaman 18...
📄 Scraping halaman 19...
📄 Scraping halaman 20...
📄 Scraping halaman 21...
📄 Scraping halaman 22...
📄 Scraping halaman 23...
📄 Scraping halaman 24...
📄 Scraping halaman 25...
✅ Data dari 25 halaman berhasil disimpan ke 'data_rumah123_2.csv'


In [43]:
df = pd.read_csv('data_rumah123.csv')
df

,KT,KM,Garasi,LT,LB,Harga
0,2.0,1.0,2.0,60,80,"Rp 1,55 Miliar"
1,6.0,4.0,2.0,242,200,"Rp 4,6 Miliar"
2,4.0,3.0,1.0,106,110,"Rp 1,8 Miliar"
3,5.0,4.0,1.0,85,220,"Rp 3,3 Miliar"
4,3.0,2.0,2.0,105,120,"Rp 2,2 Miliar"
...,...,...,...,...,...,...
495,3.0,2.0,1.0,58,78,"Rp 1,68 Miliar"
496,4.0,3.0,2.0,114,238,"Rp 3,35 Miliar"
497,3.0,3.0,1.0,90,129,"Rp 2,65 Miliar"
498,4.0,4.0,1.0,136,150,"Rp 2,99 Miliar"
